# BERT's playground 
Hello there! Welcome on BERT's playground. You may play with BERT here and see what he can do but always make sure he feels respected and admired.

## Setups


In [1]:
import pickle
import numpy as np
import tensorflow as tf
from Vectorisation import Vectorisation
from Config import Config
from BERT import MaskedLanguageModel, MaskedTextGenerator
import masking

with open("./ml4science_data.pkl", "rb") as fp:
    data_dict = pickle.load(fp)

config = Config()
vec = Vectorisation(config=config)


In [ ]:
# Prepare data for masked language model
encoded = vec.encode_dict(data_dict)
x_masked_encoded, y_masked_encoded, sample_weights = masking.mask_input_and_labels(encoded)

mlm_ds = tf.data.Dataset.from_tensor_slices((x_masked_encoded, y_masked_encoded, sample_weights))
mlm_ds = mlm_ds.shuffle(1000).batch(config.BATCH_SIZE)

# TODO: continue implementing everything downstream so that BERT can finally run freely on his playground just as he wishes
sample_tokens = vec.encode(...)
generator_callback = MaskedTextGenerator(sample_tokens)

bert_masked_model = create_masked_language_bert_model()
bert_masked_model.summary()

bert_masked_model.fit(mlm_ds, epochs=5, callbacks=[generator_callback])
bert_masked_model.save("bert_mlm_imdb.h5")